In [1]:
#This is the conjob command that will auto post images/videos to social networks
#format
#cert_cron certificate_id channel <media_type>
#example: cert_cron.py AWS_SAA LinkedIn
#PDF: 20 questions (Github)

In [2]:
SHORT_VIDEO_CHANNELS = ['Youtube', 'Facebook', 'X', 'Tiktok']   #1 video = 3 images/questions
LONG_VIDEO_CHANNELS = ['Youtube']   #1 video = 20 images/questions
IMAGE_CHANNELS = ['LinkedIn', 'PInterest', 'Reddit']    #1 image = 3 questions
PDF_CHANNELS = ['Github']   #1 PDF = 4 pages * 5 questions

In [3]:
#1 question should have flag to indicate which channel(s) it appeared. for ex: "fb": 20250528
#we should post to social network once per day

In [4]:
import sys
import os
import importlib
import pymongo
from dotenv import load_dotenv
import subprocess
import time

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Get the path to the parent directory
parent_dir = os.path.dirname(notebook_dir)

# Add the parent directory to sys.path if it's not already there
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [5]:
load_dotenv(override=True) 

import thirdparty_sdk
importlib.reload(thirdparty_sdk)    #always get latest code
import exam_settings
importlib.reload(exam_settings) #always get latest code
import const
importlib.reload(const) #always get latest code
from thirdparty_sdk.linkedin import LinkedIn
importlib.reload(thirdparty_sdk.linkedin)    #always get latest code

<module 'thirdparty_sdk.linkedin' from '/Users/sang/Documents/Source/Python/python_webscrap/cert_exam/thirdparty_sdk/linkedin.py'>

In [6]:
db_client = pymongo.MongoClient(os.environ['DB_URI'])
db = db_client['db_certificates']

In [7]:
def create_img_with_3_images(documents):
    exam_settings.generate_image_portrait_from_file('img_3_question_template.html', './', 'test_3_q.png')

#test
#create_img_with_3_images(None)

In [8]:
def get_cert_metadata(cert_symbol):
    meta_collection = db['tb_cert_metadata']
    #get metadata of the certificate
    cert_metadata = meta_collection.find_one({'symbol': cert_symbol})
    return cert_metadata

In [9]:
def query_random_questions(num_of_questions, channel, cert_metadata):
    #query random questions
    condition = {'type': 'multiple-choice', 'exported': {'$ne': 1}} #not in test course
    condition[channel] = None   #never posted in this channel
    pipeline = [
                {"$match": condition},
                {"$sample": {"size": num_of_questions}} #randomly documents
            ]
    collection = db[cert_metadata['collection_name']]
    random_documents = list(collection.aggregate(pipeline))
    if len(random_documents) < num_of_questions:
        print('Not enough questions to export')
        return []
    return random_documents

In [10]:
def update_questions_posted(cert_metadata, channel, today_yyyymmdd, docs):
    collection = db[cert_metadata['collection_name']]
    for doc in docs:
        #indicate that this question is shared in this channel at this date
        collection.update_one({'uuid': doc['uuid']}, {'$set':{channel: today_yyyymmdd}})

In [11]:
def generate_1_img_multiple_questions(random_documents, cert_metadata, today_yyyymmdd):
    question_index = 1
    document_html = []
    document_html.append(exam_settings.html_head_1_img_3_q_str)
    #1. add header
    document_html.append('<div class="header">'+cert_metadata['name']+'</div>')
    #2. add questions
    for doc in random_documents:
        container_html = []
        container_html.append('<div class="container">')
        str_index = str(question_index) + ') '
        #question first
        container_html.append('<div class="question">'+str_index + doc['question']+'</div>')
        #2.1 add answers
        container_html.append('<div class="answers">')
        for key in doc['options'].keys():
            container_html.append( f'''
                    <div class="answer">
                        <label>{key}. {doc['options'][key]}</label>
                    </div>''')
        container_html.append('</div>') #end list of answers
        container_html.append('</div>') #end 1 container
        document_html.append(''.join(container_html))
        question_index += 1
    #add footer (optional because image/video cannot click on the link)
    # document_html.append('<div class="footer">Check out more questions via <a href="'+cert_metadata['udemy_link']+'">this link</a></div>')
    #end doc
    document_html.append(exam_settings.html_tail_1_img_3_q_str)
    #1 doc 1 image
    filename = 'img_multi_q_'+today_yyyymmdd + '.png'
    creation_result = exam_settings.generate_image_portrait(''.join(document_html), cert_metadata['img_m_q_folder_path'], filename)
    return creation_result, filename

In [12]:
#create video with only 1 image
#sample: ffmpeg -loop 1 -i input_image.jpg -i input_audio.mp3 -c:v libx264 -tune stillimage -c:a aac -b:a 192k -pix_fmt yuv420p -t 10 -shortest output_video.mp4
def create_video_from_image(img_path, output_filename):
    audio_path = os.path.abspath('audio/bg_audio_16sec.m4a')
    abs_output_path = os.path.abspath(output_filename)
    ffmpeg_command = [
        "/Users/sang/Downloads/SetupFiles/ffmpeg/ffmpeg",
        "-loop", "1",
        "-i", img_path,
        "-i", audio_path,
        "-c:v", "libx264",
        "-tune", "stillimage",
        "-c:a", "aac",
        "-b:a", "192k",
        "-pix_fmt", "yuv420p",
        "-t", "10",
        "-shortest",
        "-y", # Overwrite output file without asking
        abs_output_path
    ]

    try:
        result = subprocess.run(
            ffmpeg_command,
            # cwd=abs_image_folder, # This changes the directory for THIS command execution
            check=True,
            capture_output=True,
            text=True
        )
        print(f"Video '{output_filename}' created successfully at {abs_output_path}")
        return True
    except FileNotFoundError:
        print(f"Error: FFmpeg command not found. Make sure FFmpeg is installed and in your system's PATH.")
        return False
    except subprocess.CalledProcessError as e:
        print(f"Error: FFmpeg command failed with exit code {e.returncode}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [13]:
def main(cert_symbol, channel):
    num_of_questions = 3    #default we export 3 questions
    if channel in LONG_VIDEO_CHANNELS or channel in PDF_CHANNELS:
        num_of_questions = 20
    #
    cert_metadata = get_cert_metadata(cert_symbol)
    #query random questions
    random_documents = query_random_questions(num_of_questions, channel, cert_metadata)
    if len(random_documents) == 0:
        return
    today_yyyymmdd = const.get_current_date_yyyymmdd()
    if channel in SHORT_VIDEO_CHANNELS or channel in IMAGE_CHANNELS:
        #create 1 image with multiple questions
        creation_result, img_filename = generate_1_img_multiple_questions(random_documents, cert_metadata, today_yyyymmdd)
        if creation_result == False:
            return
        if channel in SHORT_VIDEO_CHANNELS:
            #create video with 1 image only
            video_name = 'vid_multi_q_'+today_yyyymmdd + '.mp4'
            result_create_video = create_video_from_image(cert_metadata['img_m_q_folder_path']+img_filename, cert_metadata['img_m_q_folder_path'] + video_name)
            if result_create_video == False:
                return
            time.sleep(5)   #delay
            #todo: upload the video to social networks
        if channel in IMAGE_CHANNELS:   #1 image for 1 post
            answers_contents = []
            description_contents = []
            description_contents.append('#'+ cert_metadata['name'] + ' practice questions:\n')
            question_index = 1
            for doc in random_documents:
                str_index = str(question_index) + ') '
                description_contents.append(str_index + doc['question'])
                answers_contents.append(str_index + doc['answer'])
                for explanation_key in doc['explanation']:
                    answers_contents.append(explanation_key + '. ' + doc['explanation'][explanation_key])
                question_index += 1

            # description_contents.append('\n' + 'Answers and explanations are in the comment')
            description_str = '\n'.join(description_contents)
            # print(description_str)
            answers_str = '\n'.join(answers_contents)
            description_str += '\n\n' + answers_str
            # print(answers_str)
            is_success_share = False
            if channel == 'LinkedIn':
                linkedin_obj = LinkedIn(os.environ['LI_URI'], os.environ['LI_REST_URI'], os.environ['LI_ACCESS_TOKEN'], os.environ['LI_VERSION'])
                is_success_share = linkedin_obj.upload_and_share_img(cert_metadata, description_str, answers_str, cert_metadata['img_m_q_folder_path'] + img_filename)
            #update flag to shared questions
            if is_success_share == True:
                update_questions_posted(cert_metadata, channel, today_yyyymmdd, random_documents)

#test
if __name__ == '__main__':
    print(sys.argv)
    args = sys.argv
    if len(args) == 1:
        #todo there is no any param, get all certificates and post to all channels
        todo = 1

    # cert_symbol = args[1]   #certificate symbol
    # channel = args[2]
    #
    cert_symbol = 'AWS_CLF_C02' #for testing
    channel = 'LinkedIn'
    #
    main(cert_symbol, channel)

['/Users/sang/.pyenv/versions/3.9.10/lib/python3.9/site-packages/ipykernel_launcher.py', '--f=/Users/sang/Library/Jupyter/runtime/kernel-v3b3cbc10947811e5cbc8210985cb880e769fa28d9.json']


[5015:58389:0602/205324.453978:ERROR:net/cert/internal/trust_store_mac.cc:817] Error parsing certificate:
ERROR: Failed parsing extensions

[5021:58413:0602/205325.313626:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[5021:58413:0602/205325.315044:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[5021:58413:0602/205325.315744:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[5021:58413:0602/205325.504503:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[5021:58413:0602/205325.579992:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[5021:58413:0602/205325.581209:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[5021:58413:0602/205325.583049:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Erro

Done generating 1 page image
{'value': {'mediaArtifact': 'urn:li:digitalmediaMediaArtifact:(urn:li:digitalmediaAsset:D5622AQHjlIgO7NTwKQ,urn:li:digitalmediaMediaArtifactClass:uploaded-image)', 'uploadMechanism': {'com.linkedin.digitalmedia.uploading.MediaUploadHttpRequest': {'uploadUrl': 'https://www.linkedin.com/dms-uploads/sp/v2/D5622AQHjlIgO7NTwKQ/uploaded-image/B56Zcw6ypMHUAA-/0?ca=vector_feedshare&cn=uploads&iri=B01-86&sync=0&v=beta&ut=0J-aVlhYDxMHM1', 'headers': {'media-type-family': 'STILLIMAGE'}}}, 'asset': 'urn:li:digitalmediaAsset:D5622AQHjlIgO7NTwKQ', 'assetRealTimeTopic': 'urn:li-realtime:digitalmediaAssetUpdatesTopic:urn:li:digitalmediaAsset:D5622AQHjlIgO7NTwKQ'}}
Status Code: 201
Headers: {'Cache-Control': 'no-cache, no-store', 'Pragma': 'no-cache', 'Expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'Set-Cookie': 'lang=v=2&lang=en-us; SameSite=None; Path=/; Domain=linkedin.com; Secure, bcookie="v=2&f4159cd5-ccb7-414f-8efe-ba7c794bc02c"; domain=.linkedin.com; Path=/; Secure; Expi

In [14]:
#test
from thirdparty_sdk.linkedin import LinkedIn
importlib.reload(thirdparty_sdk.linkedin)    #always get latest code

cert_metadata = get_cert_metadata('AWS_SAA')
question_list = ["111111", "2222", "333"]
video_name = 'img_multi_q_20250529.png'
file_size = const.get_file_size(cert_metadata['img_m_q_folder_path'] + video_name)
# print(file_size)  #554191
linkedin_obj = LinkedIn(os.environ['LI_URI'], os.environ['LI_REST_URI'], os.environ['LI_ACCESS_TOKEN'], os.environ['LI_VERSION'])
# is_success_share = linkedin_obj.upload_and_share_video(cert_metadata, question_list, cert_metadata['img_m_q_folder_path'] + video_name, file_size)
decription = cert_metadata['name'] + ' practice questions \n' + '\n'.join(question_list)
print(decription)
# is_success_share, post_id = linkedin_obj.upload_and_share_img(cert_metadata, decription, cert_metadata['img_m_q_folder_path'] + video_name)

AWS Certified Solutions Architect - Associate (SAA-C03) practice questions 
111111
2222
333
